In [122]:
import pandas as pd

In [139]:
#import the findings
df = pd.read_csv("../hars_general_news_out.csv")
# for csv in ["../hars_entertainment_out.csv", "../hars_online_shopping_out.csv", "../hars_sports_out.csv", "../hars_sports_out.csv", "../hars_games_out.csv", "../hars_travel_out.csv"]: 
#     df = pd.concat([df,pd.read_csv(csv)],ignore_index=True)
    
# remove location related leakages 
dropIndex = df[ (df["plain_pii"]=="60201") | (df["plain_pii"]=="Illinois") | (df["plain_pii"]=="Evanston") | (df["third_party"].isna())].index
df.drop(labels=dropIndex,axis=0,inplace=True)
df.reset_index(inplace=True, drop=True)
df.head(50)


73

In [124]:
df.groupby(df["first_party"]).count()

,Unnamed: 0,pageref,third_party,http_method,startedDateTime,leak_method,encoding,plain_pii,raw_pii,where,key,value,mimeType,text,index
first_party,,,,,,,,,,,,,,,
bild.de,3,3,3,3,3,3,3,3,0,3,0,0,3,3,3
bizjournals.com,1,1,1,1,1,1,1,1,0,1,0,0,1,1,1
booking.com,18,18,18,18,18,18,18,18,0,18,18,18,0,0,18
businesswire.com,2,2,2,2,2,2,2,2,0,2,0,0,2,2,2
cbc.ca,7,7,7,7,7,7,7,7,0,7,0,0,7,7,7
chicagotribune.com,8,8,8,8,8,8,8,8,0,8,4,4,4,4,8
cnbc.com,39,39,39,39,39,39,39,39,0,39,6,6,33,33,39
corriere.it,22,22,22,22,22,22,22,22,0,22,15,15,7,7,22
economist.com,5,5,5,5,5,5,5,5,0,5,0,0,5,5,5


In [125]:
#third parties simplified to etld+1
from publicsuffixlist import PublicSuffixList
psl = PublicSuffixList()
# we only want to see the first party to same third party once
df.loc[:,"third_party"] = df.loc[:,"third_party"].map(lambda tp : psl.privatesuffix(tp))

nodups = df.drop_duplicates(subset=["first_party", "third_party"])
# nodups = df
# remove instances of customized subdomain
nodups.loc[:,"third_party"] = nodups.loc[:,"third_party"].map(lambda x : psl.privatesuffix(x)) 


/tmp/ipykernel_47465/3170461779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodups.loc[:,"third_party"] = nodups.loc[:,"third_party"].map(lambda x : psl.privatesuffix(x))


In [126]:
# how many first parties have this third party
nodups["third_party"].groupby(nodups["third_party"]).count().sort_values(ascending=False).to_csv("partners.csv")
nodups["third_party"].groupby(nodups["third_party"]).count().sort_values(ascending=False)
# nodups["third_party"].groupby(nodups["third_party"]).count().sort_values(ascending=False).aggregate('sum', axis=0)


third_party
facebook.com              14
rlcdn.com                 10
permutive.com              6
loginradius.com            3
snapchat.com               3
google-analytics.com       3
gravatar.com               3
piano.io                   3
viafoura.co                2
tinypass.com               2
tiktok.com                 2
sail-track.com             2
publicmediasignin.org      2
mparticle.com              2
amazon-adsystem.com        2
arcpublishing.com          2
htsmartcast.com            1
auth0.com                  1
scorecardresearch.com      1
segment.io                 1
sjv.io                     1
slurrp.com                 1
curbed.com                 1
spot.im                    1
thecut.com                 1
condenast.com              1
trb.com                    1
google.com                 1
vnecdn.net                 1
vulture.com                1
webengage.com              1
weborama.com               1
weborama.fr                1
www.googleapis.com         1
ya

In [127]:
#number of third parties per first party
nodups["first_party"].groupby(nodups["first_party"]).count().sort_values(ascending=False).to_csv("partners.csv")
nodups["first_party"].groupby(nodups["first_party"]).count().sort_values(ascending=False)
# nodups["first_party"].groupby(nodups["first_party"]).count().sort_values(ascending=False).aggregate('sum', axis=0)

first_party
hindustantimes.com    9
nymag.com             7
usnews.com            5
usatoday.com          5
latimes.com           4
nbcnews.com           4
rbc.ru                4
repubblica.it         3
independent.co.uk     3
nypost.com            3
news.com.au           3
mirror.co.uk          3
cnbc.com              3
wired.com             3
reuters.com           3
scmp.com              2
telegraph.co.uk       2
time.com              2
newsweek.com          2
wsj.com               2
infobae.com           2
booking.com           2
indianexpress.com     2
huffpost.com          2
cbc.ca                2
chicagotribune.com    2
express.co.uk         2
lenta.ru              1
techspot.com          1
vnexpress.net         1
businesswire.com      1
udn.com               1
thesun.co.uk          1
theatlantic.com       1
corriere.it           1
smh.com.au            1
spiegel.de            1
lefigaro.fr           1
economist.com         1
flipboard.com         1
foxnews.com           1
pbs.

In [128]:
nodups.loc[nodups['first_party'] == "hindustantimes.com"]

,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,leak_method,encoding,plain_pii,raw_pii,where,key,value,mimeType,text,index
406,1135,hindustantimes.com,https://accounts.hindustantimes.com/?htMoe=09b...,livemint.com,GET,2023-02-25T09:57:28.017Z,get,sha256,rogernadal322@gmail.com,NaN,value,ud[em],eba23eb9253d668bc24763ce39c4a3b51c5cdd8fd25b1c...,NaN,NaN,0.0
408,1137,hindustantimes.com,https://accounts.hindustantimes.com/?htMoe=09b...,facebook.com,GET,2023-02-25T09:57:34.354Z,get,sha256,rogernadal322@gmail.com,NaN,value,udff[em],eba23eb9253d668bc24763ce39c4a3b51c5cdd8fd25b1c...,NaN,NaN,0.0
416,1145,hindustantimes.com,https://images.livemint.com/ssoconnect/login.h...,webengage.com,GET,2023-02-25T09:58:02.822Z,get,plaintext,rogernadal322,NaN,value,lp,https://images.livemint.com/ssoconnect/login.h...,NaN,NaN,142.0
422,1151,hindustantimes.com,https://images.livemint.com/ssoconnect/login.h...,livehindustan.com,GET,2023-02-25T09:58:04.088Z,get,plaintext,rogernadal322,NaN,value,name,rogernadal322,NaN,NaN,0.0
437,1166,hindustantimes.com,https://images1.livehindustan.com/ssoconnect/l...,healthshots.com,GET,2023-02-25T09:58:05.482Z,get,plaintext,rogernadal322,NaN,value,name,rogernadal322,NaN,NaN,0.0
441,1170,hindustantimes.com,https://images.ottplay.com/ssoconnect/login.ht...,ottplay.com,GET,2023-02-25T09:58:06.464Z,get,plaintext,rogernadal322,NaN,value,name,rogernadal322,NaN,NaN,0.0
443,1172,hindustantimes.com,https://images.slurrp.com/ssoconnect/login.htm...,slurrp.com,GET,2023-02-25T09:58:07.501Z,get,plaintext,rogernadal322,NaN,value,name,rogernadal322,NaN,NaN,0.0
445,1174,hindustantimes.com,https://www.htsmartcast.com/ssoconnect/login.h...,htsmartcast.com,GET,2023-02-25T09:58:07.878Z,get,plaintext,rogernadal322,NaN,value,name,rogernadal322,NaN,NaN,0.0
447,1176,hindustantimes.com,https://images.htreaders.com/ssoconnect/login....,htreaders.com,GET,2023-02-25T09:58:08.228Z,get,plaintext,rogernadal322,NaN,value,name,rogernadal322,NaN,NaN,0.0


In [129]:
#number of first parties per third party
nodups.loc[nodups['first_party'] == "wired.com"]

,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,leak_method,encoding,plain_pii,raw_pii,where,key,value,mimeType,text,index
455,1217,wired.com,https://id.condenast.com/interaction/QkxhTSH3s...,condenast.com,POST,2023-02-18T05:37:12.016Z,post,plaintext,rogernadal322@gmail.com,NaN,body,NaN,NaN,application/x-www-form-urlencoded,oidcSessionInvalidate=&email=rogernadal322%40g...,29.0
495,1268,wired.com,https://www.wired.com/?redirectURL=/,pippio.com,GET,2023-02-18T05:46:00.115Z,get,md5,rogernadal322@gmail.com,NaN,value,iv,b47b636ac3fcb606572fe960f38b703f,NaN,NaN,0.0
501,1274,wired.com,https://www.wired.com/?redirectURL=/,rlcdn.com,GET,2023-02-18T05:46:00.643Z,get,md5,rogernadal322@gmail.com,NaN,value,m,b47b636ac3fcb606572fe960f38b703f,NaN,NaN,0.0


In [130]:
# piis leaked
df.groupby(df["plain_pii"]).count()


,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,leak_method,encoding,raw_pii,where,key,value,mimeType,text,index
plain_pii,,,,,,,,,,,,,,,
123Andy#!000,21,21,21,21,21,21,21,21,0,21,0,0,21,21,21
nadal,70,70,64,70,70,70,70,70,0,70,57,57,13,13,70
roger,117,117,117,117,117,117,117,117,0,117,98,98,19,19,117
rogernadal,22,22,22,22,22,22,22,22,0,22,22,22,0,0,22
rogernadal322,46,46,43,46,46,46,46,46,0,46,38,38,8,8,46
rogernadal322@gmail.com,557,557,554,557,557,557,557,557,0,557,418,418,139,139,557


In [131]:
#leak method
df.groupby(df["leak_method"]).count()


,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,encoding,plain_pii,raw_pii,where,key,value,mimeType,text,index
leak_method,,,,,,,,,,,,,,,
cookie,29,29,29,29,29,29,29,29,0,29,29,29,0,0,29
get,251,251,249,251,251,251,251,251,0,251,251,251,0,0,251
header,353,353,349,353,353,353,353,353,0,353,353,353,0,0,353
post,200,200,194,200,200,200,200,200,0,200,0,0,200,200,200


In [132]:
# which keys were used in http headers
df[df["leak_method"]=="header"]

,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,leak_method,encoding,plain_pii,raw_pii,where,key,value,mimeType,text,index
3,5,time.com,https://time.com/create-account-login,rlcdn.com,GET,2023-02-18T04:54:05.844Z,header,md5,rogernadal322@gmail.com,NaN,value,:path,/api/identity/envelope?pid=13444&it=4&iv=eba23...,NaN,NaN,163.0
4,6,time.com,https://time.com/create-account-login,rlcdn.com,GET,2023-02-18T04:54:05.844Z,header,sha256,rogernadal322@gmail.com,NaN,value,:path,/api/identity/envelope?pid=13444&it=4&iv=eba23...,NaN,NaN,41.0
5,7,time.com,https://time.com/create-account-login,rlcdn.com,GET,2023-02-18T04:54:05.844Z,header,sha1,rogernadal322@gmail.com,NaN,value,:path,/api/identity/envelope?pid=13444&it=4&iv=eba23...,NaN,NaN,114.0
13,22,nypost.com,https://nypost.com/account/login,spot.im,POST,2023-02-18T07:22:11.328Z,header,base64,roger,NaN,value,x-access-token,eyJhbGciOiJIUzI1NiJ9.eyJzdWIiOiJ1X0RnaUNoSGhRc...,NaN,NaN,145.0
14,23,nypost.com,https://nypost.com/account/login,referralsaasquatch.com,POST,2023-02-18T07:22:23.003Z,header,base64,roger,NaN,value,x-saasquatch-user-token,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJ1c2VyI...,NaN,NaN,193.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,1910,indiatimes.com,https://indiatimes.com/,facebook.com,GET,2023-02-18T06:08:18.094Z,header,sha256,rogernadal322@gmail.com,NaN,value,:path,/tr/?id=345736496016245&ev=PageView&dl=https:/...,NaN,NaN,199.0
823,1950,theatlantic.com,https://accounts.theatlantic.com/login/,facebook.com,GET,2023-02-18T06:55:59.922Z,header,sha256,rogernadal322@gmail.com,NaN,value,:path,/tr/?id=1407501962855831&ev=SubscribedButtonCl...,NaN,NaN,740.0
827,1954,theatlantic.com,https://accounts.theatlantic.com/login/,facebook.com,GET,2023-02-18T06:56:12.708Z,header,sha256,roger,NaN,value,:path,/tr/?id=1407501962855831&ev=InitiateCheckout&d...,NaN,NaN,300.0
828,1955,theatlantic.com,https://accounts.theatlantic.com/login/,facebook.com,GET,2023-02-18T06:56:12.708Z,header,sha256,nadal,NaN,value,:path,/tr/?id=1407501962855831&ev=InitiateCheckout&d...,NaN,NaN,374.0


In [133]:
#encodings

df.groupby(df["encoding"]).count()

,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,leak_method,plain_pii,raw_pii,where,key,value,mimeType,text,index
encoding,,,,,,,,,,,,,,,
base64,176,176,170,176,176,176,176,176,0,176,161,161,15,15,176
md5,78,78,78,78,78,78,78,78,0,78,69,69,9,9,78
plaintext,296,296,290,296,296,296,296,296,0,296,157,157,139,139,296
sha1,50,50,50,50,50,50,50,50,0,50,50,50,0,0,50
sha256,233,233,233,233,233,233,233,233,0,233,196,196,37,37,233


In [134]:
#find third party receiving same infromation from more than one first party
grouped = df.groupby(["third_party","plain_pii"])["first_party"].nunique().reset_index()
grouped
grouped[grouped["first_party"]>1].sort_values("first_party", ascending=False)
#.to_csv("suspected_leakers.csv")
# df.groupby(["third_party","plain_pii"]).apply(lambda x:x).reset_index()

,third_party,plain_pii,first_party
13,facebook.com,rogernadal322@gmail.com,14
70,rlcdn.com,rogernadal322@gmail.com,10
55,permutive.com,rogernadal322@gmail.com,5
10,facebook.com,nadal,4
11,facebook.com,roger,4
23,gravatar.com,rogernadal322@gmail.com,3
34,loginradius.com,123Andy#!000,3
37,loginradius.com,rogernadal322@gmail.com,3
77,snapchat.com,rogernadal322@gmail.com,3
57,piano.io,roger,3


In [135]:

grouped = df.groupby(["third_party","plain_pii"])["first_party"].apply(set).reset_index()
grouped

,third_party,plain_pii,first_party
0,amazon-adsystem.com,rogernadal322@gmail.com,"{usnews.com, usatoday.com}"
1,amazonaws.com,123Andy#!000,{mirror.co.uk}
2,amazonaws.com,rogernadal322@gmail.com,{mirror.co.uk}
3,arcpublishing.com,123Andy#!000,{infobae.com}
4,arcpublishing.com,rogernadal322@gmail.com,"{reuters.com, infobae.com}"
...,...,...,...
96,www.googleapis.com,roger,{time.com}
97,www.googleapis.com,rogernadal322@gmail.com,{time.com}
98,yahoo.com,rogernadal322@gmail.com,{huffpost.com}
99,yandex.com,rogernadal322@gmail.com,{rbc.ru}


In [136]:
#find the favored id per third party